# An Introduction to LangChain

In this notebook, we will cover the very basics of LangChain. This includes chains and templates.

First, we import necessary libraries, and use `dotenv` to load our OpenAI API key.

In [3]:
from langchain_community.chat_models import ChatOpenAI
import openai
import os

from dotenv import load_dotenv

api_key = os.getenv("OPENAI_API_KEY")

load_dotenv()

True

In [4]:
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

In [5]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key)

C:\Users\gamee_fy\AppData\Local\Temp\ipykernel_20512\1543617194.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key)


In every chat, LLMs are typically first introduced to a "system message," instructing the LLM on how to interpret the conversation. There is also a "human" or "user" message which is simply what the user sends to the LLM. An "assistant" or "AI" message is associated with the messages that the LLM itself writes. 

In [6]:
messages = [
    SystemMessage(content="You are an expert data scientist."),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data. Only return the script.")
]

response = llm(messages=messages, temperature=0.6)
print(response.content)

C:\Users\gamee_fy\AppData\Local\Temp\ipykernel_20512\2497210350.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages=messages, temperature=0.6)


```python
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 5)
y = np.random.randint(0, 2, 1000)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Build a simple neural network model
model = Sequential()
model.add(Dense(10, input_dim=5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
```


Note the use of the "temperature" parameter. Temperature is a parameter that refers to the probability with which the LLM's underlying next token predictor picks out a next token that is not the highest probability token. You can consider to it a proxy for "creativity." Higher temperature -> more randomness -> more "creativity."

We represent prompts using "prompt templates," which allow us to dynamically plug things into prompts. The `PromptTemplate` class is simply LangChain's object interface with prompts. Chains allow us to link prompts together. 

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
prompt = """
Explain {topic} in one sentence
"""

prompt = PromptTemplate.from_template(prompt)

In [9]:
chain = LLMChain(prompt=prompt, llm=llm)

chain.run(topic="atitude")

C:\Users\gamee_fy\AppData\Local\Temp\ipykernel_20512\1723417862.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm)
C:\Users\gamee_fy\AppData\Local\Temp\ipykernel_20512\1723417862.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(topic="atitude")


"Attitude refers to a person's feelings, beliefs, and behaviors towards a particular situation, person, or thing."

In [14]:
second_prompt = PromptTemplate(
    input_variables=["ml_topic_desc"],
    template="""
    You are a:
    {ml_topic_desc}
    """
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain.run(topic = "potato")

'Potato is a starchy tuber vegetable that is a staple food in many cultures around the world.'

The outputs of the first chain are passed in to the second chain as input.

In [16]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

blog_post = overall_chain.run(input="autoencoder")
print(blog_post)



> Entering new SimpleSequentialChain chain...
An autoencoder is a type of artificial neural network that learns to reconstruct input data, typically used for dimensionality reduction and feature learning.
This type of neural network is commonly used in unsupervised learning tasks such as image and text data compression, anomaly detection, and denoising. Autoencoders consist of an encoder network that compresses the input data into a lower-dimensional representation, and a decoder network that reconstructs the original input from this representation. By training the autoencoder to minimize the reconstruction error, it learns to capture the most important features of the input data in the encoded representation. This can be useful for tasks such as data visualization, pretraining for other machine learning models, and generating new data samples.

> Finished chain.
This type of neural network is commonly used in unsupervised learning tasks such as image and text data compression, anoma